<a href="https://colab.research.google.com/github/MatteoMakovec/Recommender_Systems/blob/main/Recommender_system_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## EVALUATIONS
split_percentage = 0.8
num_int = URL_all.nnz

train_mask = np.random.choice([True, False], num_int, p=[split_percentage, 1-split_percentage])
URM_train = sps.csr_matrix(URM_all.data[train_mask], (URM_all.row[train_mask], URM_all.column[train_mask]))

def precision(recommended_items, relevant_items):
  is_relevant = np.ed1d(recommended_items, relevant_items, assume_unique = False)
  precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
  return precision_score

def recall(recommended_items, relevant_items):
  is_relevant = np.ed1d(recommended_items, relevant_items, assume_unique = True)
  recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
  return recall_score

def MAP(recommended_items, relevant_items):
  is_relevant = np.ed1d(recommended_items, relevant_items, assume_unique = True)
  p_at_k = is_relevant*np.cumsum(is_relevant, dtype=np.float32) / (1+np.arange(is_relevant.shape[0]))
  map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])
  return map_score



## RANDOM RECOMMENDER: TOP-N ITEMS
def fit(self, URM_train):
  item_popularity = np.ediff1d(URM_all.tocsc().indptr)
  self.popular_items = np.argsort(item_popularity)
  self.popular_items = np.flip(self.popular_items, axis = 0)

def recommend(self, user_id, k=5):
  seen_items = self.URM_train.indices[self.URM_train.indptr[user_id]:self.URM_train.indptr[userid+1]]
  unseen_items_mask = np.in1d(self.popular_items, seen_items, assume_unique=True, invert=True)
  unseen_items = self.popular_items[unseen_items_mask]
  recommended_items = self.popular_items[0:k]
  return recommended_items



## GLOBAL EFFECTS
def fit(self, URM_train):
  global_average = np.mean(self.URM_train.data())

  # global average
  URM_train_unbiased = self.URM_train.copy()
  URM_train_unbiased.data -= global_average

  # user bias
  user_mean = URM_train_unbiased.mean(axis=1)
  user_mean = np.array(user_mean).squeeze()

  for user_id in range(len(user_mean)):
    start_position = URM_train_unbiased.indptr[user_id]
    end_position = URM_train_unbiased.indptr[user_id+1]
    URM_train_unbiased.data[start_position:end_position] -= user_mean[user_id]

  # item bias
  item_mean = URM_train_unbiased.mean(axis=0)
  item_mean = np.array(item_mean).squeeze()

  self.best_rated = np.argsort(item_mean)
  self.best_rated = np.flip(self.best_rated, axis = 0)

def recommend(self, user_id, k=10):
  seen_items = self.URM_train.indices[user_id]:self.URM_train.indices[user_id+1]
  unseen_items_mask = np.id1(self.best_rated, seen_items, invert=True)
  unseen_items = self.best_rated[unseen_items_mask]
  recommended_items = unseen_items[0:k]
  return recommended_items



## CONTENT-BASED FILTERING
class KNN_CBF(object):
  def __init__(self, ICM, URM):
    self.URM = URM
    self.ICM = ICM

  def fit(self, topK=50, shrink=100, similarity="cosine"):
    numerator = ICM_all[item_id].dot(ICM_all.T).toarray().ravel()
    item_norms = np.sqrt(np.array(ICM_all.T.power(2).sum(axis=0))).ravel()
    denominator = item_norms[item_id] * item_norms + shrink + 1e-6
    similarity_vecor = numerator / denominator
    sorted_item_indices = np.argsort(-similarity_vector)
    self.W_sparse = sorted_item_indices[0:topK]

  def recommend(self, user_id, at=None):
    user_profile = URM[user_id]
    scores = user_profile.dot(self.W_sparse).toarray().ravel()
    scores = scores[self.URM.indices[self.URM.indptr[user_id]]:self.URM.indices[self.URM.indptr[user_id+1]]]
    ranking = scores.argsort()[::-1]
    return ranking[:at]



## IDF
num_tot_items = ICM_all.shap[0]
items_per_feature = np.ediff1d(ICM_all.indptr)
IDF = np.array(np.log(num_top_items/items_per_feature))
np.sort(IDF)



## ITEM-BASED CF
class Item_CF(object):
  def __init__(self, URM):
    self.URM = URM
  
  def fit(self, topK=50, shrink=100, similarity="cosine", normalized=True):
    similarity_object = Compute_Similarity_Python(self.URM, shrink=shrink, topK=topK, similarity=similarity, normalize=normalize)
    self.W_sparse = similarity_object.compute_similarity()

  def recommend(self, user_id, at:None, filter_seen=True):
    user_profile = URM[user_id]
    scores = user_profile.dot(self.W_sarse).toarray().ravel()
    scores = self.filter_seen(user_id, scores)
    ranking = scores.argsort()[::-1]
    return rankin[:at]

  def filter_seen(self, user_id, scores):
    start_pos = self.URM.indptr[user_id]
    end_pos = self.URM.indptr[user_id+1]
    user_profile = self.URM.indice[start_pos:end_pos]
    scores[user_profile] = -np.inf
    return scores



## USER-BASED CF
class Item_CF(object):
  def __init__(self, URM):
    self.URM = URM
  
  def fit(self, topK=50, shrink=100, similarity="cosine", normalized=True):
    similarity_object = Compute_Similarity_Python(self.URM.T, shrink=shrink, topK=topK, similarity=similarity, normalize=normalize)
    self.W_sparse = similarity_object.compute_similarity()

  def recommend(self, user_id, at:None, filter_seen=True):
    scores = self.W_sparse[user_id, :].dot(self.URM).toarray().ravel()
    scores = self.filter_seen(user_id, scores)
    ranking = scores.argsort()[::-1]
    return rankin[:at]

  def filter_seen(self, user_id, scores):
    start_pos = self.URM.indptr[user_id]
    end_pos = self.URM.indptr[user_id+1]
    user_profile = self.URM.indice[start_pos:end_pos]
    scores[user_profile] = -np.inf
    return scores



## IALS
def update_row(interaction_profile, interaction_confidence, Y, YtY, regularization_diagonal):
  Y_interactions = Y[interaction_profile, :]
  A = Y_interactions.T.dot(((interaction_confidence-1)*Y_interactions.T).T)
  B = YtY + A + regularization_diagonal
  return np.dot(np.linalg.inv(B), Y_interactions.T.dot(interaction_confidence))

C_URM_train_csc = C_URM_train.tocsc()
num_factors = 10

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

VV = item_factors.T.dot(item_factors)
UU = user_factors.T.dot(user_factors)

for epochs in range (10):
  for user_id in range(C_URM_train.shape[0]):
    start_pos = C_URM_train.indptr[user_id]
    end_pos = C_URM_train.indptr[user_id+1]
    user_profile = C_URM_train.indices[start_pos:end_pos]
    user_confidence = C_URM_train.data[start_pos:end_pos]
    user_factors[user_id, :] = update_row(user_profile, user_confidence, item_factors, VV, regularization_diagonal)
  
  for item_id in range(C_URM_train.shape[1]):
    start_pos = C_URM_train_csc.indptr[item_id]
    end_pos = C_URM_train_csc.indptr[item_id+1]
    item_profile = C_URM_train_csc.indices[start_pos:end_pos]
    item_confidence = C_URM_train_csc.data[start_pos:end_pos]
    user_factors[item_id, :] = update_row(item_profile, item_confidence, user_factors, UU, regularization_diagonal)

predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])

  

## Pure SVD
U, Sigma, VT = randomized_svd(URM_train, n_components=n_factors)
user_factors = np.dot(U, np.diag(Sigma))
item_factors = VT
predicted_rating = np.dot(user_factors[user_id, :], item_factors[:, item_id])



## Item SVD
itemitem_similarity = np.dot(VT.T, VT)
predicted_rating_similarity = URM_train[user_id, :].dot(itemitem_similarity[:, item_id])






SyntaxError: ignored